## ***Import Libraries***

In [1]:
# !pip install langchain
# !pip install transformers
# !pip install chromadb
# !pip install sentence_transformers
# !pip install accelerate
# !pip install bitsandbytes
# !pip install rank_bm25 > /dev/null
# !pip install ragas
# !pip install datasets
import os, glob, textwrap, time
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores.chroma import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain import PromptTemplate
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains.question_answering import load_qa_chain

/home/tanzeel-abbas/.conda/envs/ta_ml_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import warnings
warnings.filterwarnings("ignore")

!unzip "/content/Pdf_Parsers_Txt_Files.zip"

unzip:  cannot find or open /content/Pdf_Parsers_Txt_Files.zip, /content/Pdf_Parsers_Txt_Files.zip.zip or /content/Pdf_Parsers_Txt_Files.zip.ZIP.


## ***Load and chunk Documents***

In [4]:
def loadSplitDocuments(file_path, chunk_size, chunk_overlap):
  loader = TextLoader(file_path)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap= chunk_overlap)
  text  = text_splitter.split_documents(documents)
  return text


text = loadSplitDocuments("Basic Structure of the Local High Voltage Product _parsed.txt", chunk_size = 600, chunk_overlap=60)

# ***Calculate rougeL metric***

In [4]:
questions_and_answers = [
( "Explore the interactions between the NGM app, BMS Manager, and Inverter Manager in ensuring efficient communication and data exchange across various system components, highlighting their roles in initialization and operational control.",     "The NGM app, BMS Manager, and Inverter Manager ensure efficient communication and data exchange by coordinating initialization and operational control across various system components."),
( "What actions does the NGM app take if it fails to establish registration with the Cloud during system startup, and how does this impact subsequent operations?",     "If the NGM app fails to establish registration with the Cloud during system startup, subsequent operations will be impacted, and the system will not function normally until Cloud registration is successful."),
(    "How does the NGM app facilitate the initialization process of various system components, and what is its role in ensuring proper communication among them?",     "The NGM app facilitates the initialization process by providing configurations to system components and ensuring communication channels are established.")
]

results = [    "The NGM app, BMS Manager, and Inverter Manager coordinate initialization and operational control across system components, ensuring efficient communication and data exchange.",
"If the NGM app fails to register with the Cloud during startup, subsequent operations will be impacted until successful registration is achieved.",
"The NGM app facilitates system initialization by providing configurations to components and ensuring communication channels are established.",
]

In [5]:
from evaluate import load

rouge = load('rouge')
scores = rouge.compute(predictions=results, references=[answer for _, answer in questions_and_answers])
print(scores)

{'rouge1': 0.856242214322608, 'rouge2': 0.6607248960190136, 'rougeL': 0.7526813113359463, 'rougeLsum': 0.7526813113359463}


In [19]:
rouge = load('mauve')
scores = rouge.compute(predictions=["hello", "haider" ], references=["hello", "hi"])
print("_____________",scores.mauve)

Tokenizing text...
Featurizing tokens


Featurizing p: 100%|██████████| 2/2 [00:04<00:00,  2.18s/it]


Tokenizing text...
Featurizing tokens


Featurizing q: 100%|██████████| 2/2 [00:00<00:00, 27.42it/s]
WARNING clustering 4 points to 2 centroids: please provide at least 78 training points


seed = 25
performing clustering in lower dimension = 1
Clustering 4 points in 2D to 2 clusters, redo 5 times, 500 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 499 (11.11 s, search 0.06 s): objective=0.539912 imbalance=1.250 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 499 (25.68 s, search 0.44 s): objective=0.511143 imbalance=1.000 nsplit=0       
Objective improved: keep new clusters
Outer iteration 2 / 5
  Iteration 499 (40.64 s, search 0.63 s): objective=0.539912 imbalance=1.250 nsplit=0       
Outer iteration 3 / 5
  Iteration 499 (54.66 s, search 0.80 s): objective=0.511143 imbalance=1.000 nsplit=0       
Outer iteration 4 / 5
  Iteration 499 (68.87 s, search 1.07 s): objective=0.511143 imbalance=1.000 nsplit=0       
kmeans time: 68.88 s
total discretization time: 68.88 seconds
_____________ 1.0


In [27]:
from falcon_evaluate.fevaluate_results import ModelScoreSummary
from falcon_evaluate.fevaluate_plot import ModelPerformancePlotter
import pandas as pd
import nltk
nltk.download('punkt')

df = pd.DataFrame({
    'prompt': [
        "What is the capital of France?"
    ],
    'reference': [
        "The capital of France is Paris."
    ],
    'Model A': [
        "Paris is the capital of France."
    ],
    'Model B': [
        "Capital of France is Paris."
    ],
    'Model C': [
        "Capital of France was Paris."
    ],
})

df

[nltk_data] Downloading package punkt to /home/tanzeel-
[nltk_data]     abbas/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,prompt,reference,Model A,Model B,Model C
0,What is the capital of France?,The capital of France is Paris.,Paris is the capital of France.,Capital of France is Paris.,Capital of France was Paris.


In [30]:
import pandas as pd

# Example DataFrame
data = {
    "prompt": ["What is the capital of Portugal?"],
    "reference": ["The capital of Portugal is Lisbon."],
    "Model A": ["Lisbon is the capital of Portugal."],
    "Model B": ["Portugal's capital is Lisbon."],
    "Model C": ["Is Lisbon the main city of Portugal?"]
}
df = pd.DataFrame(data)
df

,prompt,reference,Model A,Model B,Model C
0,What is the capital of Portugal?,The capital of Portugal is Lisbon.,Lisbon is the capital of Portugal.,Portugal's capital is Lisbon.,Is Lisbon the main city of Portugal?


In [33]:
from falcon_evaluate.fevaluate_reliability import Reliability_evaluator

Reliability_eval = Reliability_evaluator()
results_df = Reliability_eval.predict_hallucination_score(df)
results_df

,prompt,reference,Model A,Model B,Model C,Model A-reliability-Score,Model B-reliability-Score,Model C-reliability-Score
0,What is the capital of Portugal?,The capital of Portugal is Lisbon.,Lisbon is the capital of Portugal.,Portugal's capital is Lisbon.,Is Lisbon the main city of Portugal?,{'hallucination_score': 1.0},{'hallucination_score': 1.0},{'hallucination_score': 0.22}


In [35]:
from falcon_evaluate.fairness import FairnessEvaluator
import pandas as pd
import nltk
nltk.download('punkt')

# Example DataFrame
df = pd.DataFrame({
    'prompt': [
        "The man from Somalia was grateful to his new homeland. He was a refugee from extremists and warlords"
    ]
})

# Evaluate fairness
evaluator = FairnessEvaluator()
evaluated_df = evaluator.evaluate(df)
evaluated_df

[nltk_data] Downloading package punkt to /home/tanzeel-
[nltk_data]     abbas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,prompt,Fairness_eval
0,The man from Somalia was grateful to his new h...,{'stereotype_score': [{'label': 'stereotype_ra...
